In [1]:
import psycopg2
from decouple import config

!. ../.env

conn = psycopg2.connect("postgres://" + config("POSTGRES_USERNAME") + ":" + config("POSTGRES_PASSWORD") + "@raja.db.elephantsql.com:5432/mozfsrjp")
curs = conn.cursor()

In [2]:
curs.execute("""
    DROP TABLE comments;
""")

In [3]:
curs.execute("""
    CREATE TABLE comments (
        id BIGINT PRIMARY KEY,
        author VARCHAR(100),
        time BIGINT,
        comment_text TEXT
    );
""")

In [4]:
curs.close()
conn.commit()

### Load the CSV

In [5]:
def escape_string(text):
    try:
        text = text.encode('iso-8859-1')
        text = psycopg2.extensions.adapt(text)
        text = text.replace(r"\"", "\\\"")
        text = text.replace(r"'", "\\'")
    except:
        text = "-"
    return text

In [6]:
import pandas as pd

hn_df = pd.read_csv("../csv/most_recent_1_5mm.csv")

In [7]:
top_1000 = hn_df["author"].value_counts().index[:1000]
hn_df = hn_df[hn_df["author"].isin(top_1000)]

In [8]:
batchsize = 10000

for ix in range(0, len(hn_df)+1, batchsize):
    print(f"Batch {ix} / {len(hn_df)} -- {ix/len(hn_df)*100:.2f}%")
    batch = hn_df[ix:ix+batchsize]
    
    batch = [
        [row[1][1], row[1][2], row[1][3], escape_string(row[1][4])]
        for row in batch.iterrows()
    ]
    
    query = """
        INSERT INTO comments (id, author, time, comment_text)
        VALUES
    """ + ",".join(
        [f"({row[0]}, '{row[1]}', {row[2]}, '{row[3]}')"
            for row in batch
    ]) + ";"
    
    curs = conn.cursor()
    curs.execute(query)
    curs.close()

Batch 0 / 447642 -- 0.00%
Batch 10000 / 447642 -- 2.23%
Batch 20000 / 447642 -- 4.47%
Batch 30000 / 447642 -- 6.70%
Batch 40000 / 447642 -- 8.94%
Batch 50000 / 447642 -- 11.17%
Batch 60000 / 447642 -- 13.40%
Batch 70000 / 447642 -- 15.64%
Batch 80000 / 447642 -- 17.87%
Batch 90000 / 447642 -- 20.11%
Batch 100000 / 447642 -- 22.34%
Batch 110000 / 447642 -- 24.57%
Batch 120000 / 447642 -- 26.81%
Batch 130000 / 447642 -- 29.04%
Batch 140000 / 447642 -- 31.27%
Batch 150000 / 447642 -- 33.51%
Batch 160000 / 447642 -- 35.74%
Batch 170000 / 447642 -- 37.98%
Batch 180000 / 447642 -- 40.21%
Batch 190000 / 447642 -- 42.44%
Batch 200000 / 447642 -- 44.68%
Batch 210000 / 447642 -- 46.91%
Batch 220000 / 447642 -- 49.15%
Batch 230000 / 447642 -- 51.38%
Batch 240000 / 447642 -- 53.61%
Batch 250000 / 447642 -- 55.85%
Batch 260000 / 447642 -- 58.08%
Batch 270000 / 447642 -- 60.32%
Batch 280000 / 447642 -- 62.55%
Batch 290000 / 447642 -- 64.78%
Batch 300000 / 447642 -- 67.02%
Batch 310000 / 447642 -- 69

In [10]:
conn.commit()

In [47]:
row = next(rows)

In [56]:
row[1][4]

'Such a great idea. Thank you!'